## Explorando propiedades de temperatura finita de los materiales mediante computadoras cuánticas
### Ideas preliminares
<ul>
<li>Las propiedades térmicas de los materiales es fundamental en la comprensión de los sistemas de materia condensada.</li>
<li>Las propiedades térmicas permiten el desarrollo de nuevos materiales con aplicaciones potenciales.</li>
<li>Las computadoras clásicas no son capaces de procesar los efectos cuánticos a nanoescala. Las computadoras cuánticas pueden encargarse de ello eficientemente.</li>
<li>Los algoritmos cuánticos actuales para el estudio de las propiedades térmicas de materiales de interés son costosos computacionalmente.</li>
<li>Los estados cuánticos puros térmicos (TPQ) son una eficiente alternativa para estimar las propiedades termicas de los materiales cuánticos. No requieren la preparación del estado térmico completo ni una cantidad de muestras creciente según el tamaño del sistema.</li>
</ul>

### Objetivos
<ul>
<li>Desarrollar algoritmo para preparar estados TPQ canónicos en computadora cuánticas.</li>
<li>Implementamos circuitos cuánticos usando dicho algoritmo.</li>
<li>Verificamos la capacidad de estimación de las propiedades térmicas de los materiales cuánticos.</li>
</ul>

### Proyección
<ul>
<li>Este algoritmo de preparación de estados TPQ permitirá explorar materiales cuánticos a temperatura finita, en ordenadores cuánticos, y a corto plazo.</li>
</ul>

### Introducción

El estudio de las propiedades térmicas de los nanomateriales es de interés para la nanoelectrónica y otras potenciales aplicaciones. A tales escalas (nanométricas), los efectos cuánticos dominan. Esta predominancia del comportamiento cuántico, hace que la simulación de estos materiales superen las capacidades de incluso las más potentes supercomputadoras clásicas.

Las computadoras cuánticas son capaces de simular eficientemente sistemas cuánticos de muchos cuerpos, por lo que son una alternativa para superar las limitancias de los ordenadores clásicos en el estudio de los materiales cuánticos.

Reciéntemente se ha logrado simular sistemas a temperatura cero en computadoras cuánticas, sin embargo, todavía escasean algoritmos cuánticos para estudiar propiedades a temperatura finita de los materiales cuánticos.

<ul>
<li> El principal desafío en la exploración de propiedades a temperatura finita radica en la preparación de estados térmicos.</li>
</ul>

Los actuales algoritmos cuánticos para la preparación de estados se dividen en dos categorías



#### Primera categoría
<ul>
<li>Aquellos algoritmos que inicializan los qubits en el estado térmico completo (mixto). En este marco, que es de naturaleza variacional, el promedio térmico de un observable puede ser obtenido directamente midiendo el observable en ese estado.</li>
<li>La naturaleza variacional de estos algoritmos necesitan de una función costo dificil de calcular a medida que el sistema incrementa de tamaño, por lo que son complicados de escalar a sistemas complejos.</li>
<li>El número de qubits a usar también escala con la complejidad del sistema, esto limita el tamaño de los sistemas que pueden ser simulados con el hardware cuántico actual.</li>
</ul>

#### Segunda categoría
<ul>
<li>Preparan un conjunto de estados puros, un estado puro a la vez, donde cada estado puro ha sido muestreado según la correcta distribución térmica.</li>
<li>Para calcular promedios térmicos, el observable de interés se mide en cada uno de los diferentes estados puros, y luego se promedian en conjunto.</li>
<li>Como los estados puros son mucho más fáciles de preparar en una computadora cuántica que los estados mixtos, este marco de preparación de estados térmicos es más prometedor para las computadoras NISQ.</li>
<li>El número de muestras necesarias también suele crecer con el tamaño del sistema que se está simulando, lo cual genera requisitos de recursos para sistemas grandes.</li>
</ul>

Los estados TPQ canónicos permiten estimar promedios térmicos en computadoras cuánticas. Estos sirven también para aproximar estados térmicos (definidos en una tercera categoría) sin incurrir en recursos cuánticos para preparar un estado mixto ni depender de una cantidad de muestras creciente con el tamaño del sistema.

Los estados TPQ son formados por aplicación de una transformación no unitaria específica (que es función del hamiltoniano del sistema y de la temperatura inversa) a un estado aleatorio. El estado resultante representa el equilibrio térmico en el sentido de que los observables medidos en este estado se aproximan a los promedios térmicos. El error en el estado esperado del observable está limitado por una disminución exponencial sobre el tamaño del sistema (N).

Así, para N suficientemente grande, el valor esperado de un observable en un solo estado TPQ producirá una aproximación muy cercana al promedio térmico.

### Fundamento teórico

Se define el estado canónico TPQ de un sistema de tamaño N, gobernado por un hamiltoniano $H$ a la temperatura inversas $\beta$ como:

\begin{equation}
| \beta , N \rangle = \hat{Q} | \Psi_{R} \rangle = e^{-\beta H/2} | \Psi_{R} \rangle
\end{equation}

Donde $| \Psi_{R} \rangle = \sum_{i=1}^{2^N} c_i |i\rangle $ es un estado aleatorio, con $ |i\rangle $ una base ortonormal arbitraria. 

Definido de esta manera, $| \Psi_{R} \rangle $ ***se caracteriza como un estado puro aleatorio de Haar***. Definimos:

\begin{equation}
\langle \hat{A} \rangle ^{ens}_{\beta, N} = \dfrac{\mathrm{Tr}[ e^{-\beta H}\hat{A} ]}{\mathrm{Tr}[e^{-\beta H}]}
\end{equation}

Como el valor esperado del ensemble del operador $\hat{A}$, para un sistema según $N$ y $\beta$. 

También se define:

\begin{equation}
\langle \hat{A} \rangle ^{TPQ}_{\beta, N} = \dfrac{\langle \beta, N |\hat{A} |\beta, N \rangle}{\langle \beta, N || \beta, N\rangle}
\end{equation}

Como el valor esperado de $\hat{A}$ en un solo estado TPQ.

Se nota que $\hat{A}$ debe ser un polinomio de grado bajo de operadores locales para que se cumpla el análisis anterior. Pero este grupo contiene muchos observables significativos, incluyendo la energía, magnetización y un número de funciones correlaciones relevantes.

El error entre $\langle \hat{A} \rangle ^{TPQ}_{\beta, N}$ y $\langle \hat{A} \rangle ^{ens}_{\beta, N}$ está acotado por un valor que decae exponencialmente según aumenta el tamaño del sistema $N$. Así, para $N$ suficientemente grande, medir el observable deseado para un solo estado TPQ proporcionará una buena aproximación del promedio térmico verdadero. Para valores de $N$, la fidelidad puede incrementarse promediando sobre los valores medidos de múltiples estados TPQ.

#### Lo siguiente corresponde a la preparación de $| \Psi_{R} \rangle$

El primer paso para preparar un estado TPQ en una computadora cuántica es la preparación de $| \Psi_{R} \rangle$. Podemos aproximar dicho estado aleatorio de Haar usando circuitos cuánticos aleatorios.

Los circuitos aleatorios a considerar están compuestos por bloques, donde cada bloque está compuesto por una capa de compuertas de rotación para un solo qubit, en todos los qubits. Esto está seguido por una capa de compuertas de entrelazamiento para dos qubits. 

Las compuertas de rotación para un solo qubit son seleccionadas por un conjunto finito $\mathcal{A} = \{ RX(\dfrac{\pi}{2}),RY(\dfrac{\pi}{2},T \}$, con la restricción de que no puede seleccionarse una compuerta para el mismo qubit en dos bloques consecutivos. Es decir, si $R_{i,j}$ es la $j$-ésima compuerta de un solo qubit actuante en el qubit $i$, entonces $R_{i,j} \in \mathcal{A} \backslash R_{i,j-1}$

Por otro lado, para las compuertas de dos qubits respetan que, dada una dimensión del sistema (del hamiltoniano) de $k$, habrá $2k$ patrones de capas de compuertas de dos qubits a recorrer, asegurando todas las direcciones de acoplamiento.

Estos circuitos pueden definirse por un solo parámetro $d$ que indique el número de bloques y controle la profundidad del circuito; y pueden verse como aproximaciones sucesivas a $t$-diseños según $d$ incremente.

Un $t$-diseño unitario es una aproximación de un unitario aleatorio de Haar el cual simula precisamente los primeros $t$ momentos de los unitarios aleatorios de Haar. Grandes órdenes de $t$-diseños generan estados cuyas propiedades convergen a esos estados aleatorios de Haar. Podemos ver esta convergencia ploteando la entropía del estado aleatorio resultante.

Dicho ploteo es útil para determinar qué tan grande $d$ se necesita para un dado tamaño de sistema. El $d$ necesario serpa el punto el cual el estado de entroía converge lo suficiente con el valor $\ln{N}-1+\gamma$ característico de los estados aleatorios de Haar, con $\gamma = 0.577$ la constante de Euler-Mascheroni.

Un circuito aleatorio de $N$ qubits con $d$ capas tiene $Nd$ compuertas de rotación de un solo qubit, y $O(Nd)$ compuertas de entrelazamiento de dos qubits.

#### Esto corresponde a la generación del estado TPQ

Una vez que el estado aleatorio $| \Psi_{R} \rangle$ ha sido generado, un operador no unitario $\hat{Q}$ debe ser aplicado para generar un estado canónico TPQ. 

*Dado que las computadoras cuánticas solo pueden realizar operaciones unitarias, debe implementarse una aproximación unitaria de la transformación no unitaria basada en los recursos cuánticos disponibles. Se demostrará algoritmos con tres diferentes enfoques: QITE, un enfoque de operador dilatado y el método de FABLE.*

**Con QITE** no se requieren qubits auxiliares (ancilla) ni postselección de resultados. Esto hace necesario circuitos más profundos costosos en recursos clásicos. El tiempo de generación de circuitos con QITE aumenta rápidamente según se incrementa el tamaño del sistema simulado, lo cual puede prevenirse acudiendo al **método QITE inexacto** mediante un truncamiento del tamaño del dominio en condiciones de longitudes de correlación promedio pequeñas en comparación con el dominio seleccionado. Sin embargo, esto último reduce la precisión.

**El método de dilatación unitaria** requiere un único qubit ancilla y postselección de resultados. Se descartan los experimentos donde el qubit ancilla se encuentre en el estado $1$, esto aumenta el número de disparos requeridos para obtener buenos resultados. Este método genera circuitos menos profundos que QITE, pero requiere más disparos.

**FABLE** requiere un registro de qubits auxiliares que crece linealmente con el tamaño del sistema, además de la postselección de resultados. Para un tamaño de sistema de $N$, FABLE requiere $N+1$ qubits auxiliares, los cuales deben medirse en el estado $0$, de lo contrario el resultado debe ser descartado. Este método también necesita un número muy grande de disparos. ***Es el más favorable en términos de profundidad del circuito y tiempo de generación.***  

A saber:

![Alt text](image.png)

Nótese que en primer lugar se construye el circuito $d$-capa para generar $| \Psi_{R} \rangle$. Las capas de las compuertas de rotación de un solo qubit elegidas al azar son alternadas con las capas de compuertas de entrelazamiento de dos qubits siguiendo un patrón establecido por la dimensión del hamiltoniano ($k$). Luego, una aproximación unitaria de la transformación no unitaria $\hat{Q}$ es aplicado. El algoritmo es flexible en cómo esta operación no unitaria es implementada. El observable de interes es luego medido en el resultante estado TPQ, y es procesado y repetido R veces para construir y medir R distintos TPQ estados. Finalmente se toma el promedio de los valores térmicos medidos.

***Si se elige QITE para implementar la línea 10***, el operador no unitario se ve como una evolución a través del *tiempo imaginario*. Para la aproximación unitaria, *esta evolución se divide en pasos de tiempo imaginario*, y cada paso de tiempo se aproxima mediante un subcircuito unitario. *A mayor número de pasos, mayor fidelidad/precisión*. El tamaño del circuito cuántico crece con el número de pasos de tiempo. El subcircuito para cada paso se construye basado en un llamado *tamaño de dominio*, menor o igual que el tamaño del sistema y elegido según la longitud de correlación promedio del sistema. La precisión de QITE se mejora al aumentar este *tamaño de dominio*, pero el costo computacional clásico crece exponencionalmente con dicho tamaño.





